# Logo dimension calculator-inator

The script below takes as input the desired combined circumference of all the elements of the logo (leds_len). It will output the dimensions of each part needed to create the logo, as well as how many instances of each part are needed. 

The dimensions below are taken from TesLAN-logo-reworked--with-dimensions.svg, and are stored in a list of instructions (consisting of either parts or turns). Parts are always leading with "A", turning is leading with either "L" or "R" followed by (180-angle of turn). This may seem unordinary, but these angles are already present in the SVG file so it was simply a matter of copy-pasting. The default angle is 90 degrees. 

The dimensions can be easily verified by running the second cell, which will run a Turtle across the screen to draw the TESLAN logo in all of its glory.

In [6]:
#Letter calculations!

# F is for friends who do stuff together
# U is for you and me
# N is for anywhere and anytime at all
# Down here in the deep blue sea
# F is for fire that burns down the whole town
# U is for uranium, bombs
# N is for no survivors when you're-
# Plankton, those things aren't what fun is all about

#importing important imports
import pandas as pd
from collections import Counter

#Config
leds_len = 80 #meters (maybe put this value a little bit higher than it is in reality, so that there
              #is some slack in the LED-string (better have too many LEDS than too little))


m = {} #Measurements from SVG
#Character dimensions
m['AA'] = 173
m['AB'] = 43
m['AC'] = 62.5
m['AD'] = 209
m['AE'] = 48
m['AF'] = 125
m['AG'] = 61.5
m['AH'] = 252
m['AI'] = 147.5
m['AJ'] = 72
m['AK'] = 124
m['AL'] = 104.5
m['AM'] = 77
m['AN'] = 152
m['AO'] = 170
m['AP'] = 381.436
m['AQ'] = 187
m['AR'] = 275.465
m['AS'] = 23.1
#Inter-character spaces
m['BA'] = 35
m['BB'] = 18
m['BC'] = 234
m['BD'] = 77
m['BE'] = 48
m['BF'] = 43
m['BH'] = 118
#Total width and height
m['tw'] = 1448
m['th'] = 709.2

#angles
a={}
a['aa'] = 70.688
a['ab'] = 109.312
a['ac'] =141.376
a['ad'] =a['ac']/2 #Needed for drawing only 
a['af']= 27
a['ag'] = 117
a['lr'] = 90

def calculate_omtrek(l): #Simply add the length of each part and ignore angles and turns
    omtrek=0
    for i in l:
        if (i[0] == 'R' or i[0]=='L'):
            #skip
            pass
        else:
            omtrek+=m[i]
    return omtrek

l={} #List of instructions per letter
l['T'] = ['AA','R','AB','R','AC','L','AD','R','AE','R','AD','L','AC','R','AB','R']
l['E'] = ['AA','R','AB','R','AF','L','AG','L','AF','R','AB','R','AF','L','AG','L','AF','R','AB','R','AA','R','AH','R']
l['S'] = ['AA','R','AB','R','AF','L','AG','L','AF','R','AI','R','AA','R','AB','R','AF','L','AG','L','AF','R','AI','R']
l['L'] = ['AE','R','AD','L','AF','R','AB','R','AA','R','AH','R']
l['A_outer'] = ['AA','R','AH','R','AE','R','AL','L','AM','L','AL','R','AE','R','AH','R']
l['A_inner'] = ['AM','R','AG','R','AM','R','AG','R']
l['N']=['AE','Rag','AO','Laf','AN','R','AE','R','AH','R','AE','Rag','AO','Laf','AN','R','AE','R','AH','R']
l['chevy'] = ['L','Raa','AK','Lac','AK','Raa','AJ','Rab','AK','Rac','AK','Rab','AJ','R']
l['grootshoofd_outer'] = ['L','Rad','AP','Rab','AQ','Rab','AP','Rac','AP','Rab','AQ','Rab','AP','Rad','L']
l['grootshoofd_inner'] =['L','Rad','AR','Rab','AS','Rab','AR','Rac','AR','Rab','AS','Rab','AR','Rad','L']

#Find total circumference of each character in the logo
omtrek_pt = sum([calculate_omtrek(l['T']),
                 calculate_omtrek(l['E']),
                 calculate_omtrek(l['S']),
                 calculate_omtrek(l['L']),
                 calculate_omtrek(l['A_outer']),
                 calculate_omtrek(l['A_inner']),
                 calculate_omtrek(l['N']),
                 3*calculate_omtrek(l['chevy']),
                 calculate_omtrek(l['grootshoofd_outer']),
                 calculate_omtrek(l['grootshoofd_inner'])])

#Calculate total width, which should be equal to m['tw']. (sanity check)
total_width = 5*m['AA']+4*m['BA']+2*m['BB']+m['BC']+2*m['AE']+m['BD']
if (total_width!=m['tw']): 
    raise Exception("Huilon, the calculated width does not correspond to the given total width")
p_m = m.copy() #physical dimensions

#Update physical dimensions based on desired LED-string length
scaling_factor = leds_len/omtrek_pt
p_m.update((x, y*scaling_factor) for x, y in p_m.items())  #p_m is now in meters!


#For turtle in next cell
m_draw = m.copy()
m_draw.update((x, y*0.6) for x, y in m_draw.items()) #scale logo for turtle so it fits on screen.


#Generate a boodschappenlijstje:
l2 = [item for sublist in l.values() for item in sublist]
l2=Counter(l2)

def count_angle(key): #Add similar angles together (i.e. aa = Raa + Laa)
    global l2;
    l2[key] = l2['L'+str(key)] + l2['R'+str(key)]
    l2.pop('L'+str(key), None)
    l2.pop('R'+str(key), None)
    
#Do that a bunch of times    
count_angle('aa')
count_angle('ab')
count_angle('ac')
count_angle('ad')
count_angle('af')
count_angle('ag')
l2["lr"] = l2['L'] + l2['R']
#Remove 90 degree turns
l2.pop("L",None)
l2.pop("R",None)

#remove angle ad
l2['ac'] = l2['ac'] + l2['ad']/2 #because angle ad is only used for drawing, we need to replace it with ac
l2.pop('ad')

#Sort alphabetically
l2 = dict(sorted(l2.items()))

#Create new dictionary with both lengths and angles
c={**p_m, **a}
for key in ["ad","BA","BB","BC","BD","BE","BF","BH","tw","th"]: #Remove unwanted keys
    c.pop(key)

#Throw it all into a dataframe!
df = pd.DataFrame([l2])
df = df.append(c, ignore_index=True)
df=df.transpose()
df=df.rename({0: 'count', 1: 'Length (m)'}, axis=1) 
#Some beun to get two dataframes, one for lengths and one for angles
output=[]
for i in list(c.keys()): 
    if i[0]=="A":
        output.append(True)
    else:
        output.append(False)
output2=[not elem for elem in output]
df2=df.copy()
df2=df2.rename_axis('Part', axis=1)
df=df.rename_axis('Part', axis=1)
df=df[output]
df2=df2[output2]
df2=df2.rename({'count': 'count', 'Length (m)':'angle (deg)'}, axis=1)

#Print results
print("For the desired LED-string length, the scaling factor is:" + str(scaling_factor))
print("Below is the shopping list:")
display(df)
display(df2)


print("The total width of the logo (including spacing) is "+str(p_m['tw']) + " meters.")
print("The total height of the logo (including spacing) is "+str(p_m['th']) + " meters")

For the desired LED-string length, the scaling factor is:0.006642419620910471
Below is the shopping list:


<ipython-input-6-5fda5f548ec0>:150: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(c, ignore_index=True)


Part,count,Length (m)
AA,7.0,1.149139
AB,8.0,0.285624
AC,2.0,0.415151
AD,3.0,1.388266
AE,8.0,0.318836
AF,9.0,0.830302
AG,6.0,0.408509
AH,6.0,1.673890
AI,2.0,0.979757
AJ,2.0,0.478254


Part,count,angle (deg)
aa,2.0,70.688
ab,10.0,109.312
ac,6.0,141.376
af,2.0,27.000
ag,2.0,117.000
lr,62.0,90.000


The total width of the logo (including spacing) is 9.618223611078362 meters.
The total height of the logo (including spacing) is 4.710803995149706 meters


In [7]:
m_draw

{'AA': 103.8,
 'AB': 25.8,
 'AC': 37.5,
 'AD': 125.39999999999999,
 'AE': 28.799999999999997,
 'AF': 75.0,
 'AG': 36.9,
 'AH': 151.2,
 'AI': 88.5,
 'AJ': 43.199999999999996,
 'AK': 74.39999999999999,
 'AL': 62.699999999999996,
 'AM': 46.199999999999996,
 'AN': 91.2,
 'AO': 102.0,
 'AP': 228.86159999999998,
 'AQ': 112.2,
 'AR': 165.27899999999997,
 'AS': 13.860000000000001,
 'BA': 21.0,
 'BB': 10.799999999999999,
 'BC': 140.4,
 'BD': 46.199999999999996,
 'BE': 28.799999999999997,
 'BF': 25.8,
 'BH': 70.8,
 'tw': 868.8,
 'th': 425.52000000000004}

In [21]:
#DRAW LOGO TO SVG FILE WITH DIMENSIONS IN METERS.

#from turtle import *
from svg_turtle import SvgTurtle

#import turtle
global p, omtreks;
p=0
q=False
omtreks=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
def my_forward(n):
    global p, omtreks,q
    if (t.pen()['pendown']):
        omtreks[p]+=n
        t.forward(n/2)
        angle = t.heading()
        wz = t.width()
        t.width(0.5)
        t.color('black','black')
        t.setheading(45)
        t.heading()
        t.forward(5)
        t.right(90)
        t.write(str(round(n/0.6*scaling_factor*1000)/1000) + "m", font=("Arial", 2, "normal"))
        t.right(90)
        t.forward(5)
        t.setheading(angle)
        t.heading()
        t.color('purple','purple')
        t.width(wz)
        t.forward(n/2)
        q=False
    elif (q==False):
        q=True
        p+=1;
        t.forward(n)
    else:
        t.forward(n)
m=m_draw
def draw_letter(l):
    global a,t;
    #begin_fill()
    for i in l:
        if (i[0] == 'R' or i[0]=='L'):
            if i[0]=='R':
                if len(i[1:])>0:
                    t.right(180-float(a[i[1:]]))
                else:
                    t.right(90)
            else:
                if len(i[1:])>0:
                    t.left(180-float(a[i[1:]]))
                else:
                    t.left(90)
        else:
            my_forward(float(m[i]))
    #end_fill()

t = SvgTurtle(1000, 1000)
t.speed(10)
#turtle.tracer(0, 0)
t.penup()
t.goto(-450,200)
t.pendown()    
t.color('purple', 'purple')
draw_letter(l['T'])
t.penup()
my_forward(m['AA']+m['BA'])
t.pendown()
draw_letter(l['E'])
t.penup()
my_forward(m['AA']+m['BA'])
t.pendown()
draw_letter(l['S'])
t.penup()
my_forward(m['AA']+2*m['BB']+m['BC'])
t.pendown()
draw_letter(l['L'])
t.penup()
my_forward(m['AA']+m['BA'])
t.pendown()
draw_letter(l['A_outer'])
t.penup()
my_forward(m['BE'])
t.right(90)
my_forward(m['BF'])
t.left(90)
t.pendown()
#t.color('putple', 'white')
draw_letter(l['A_inner'])
t.penup()
t.left(90)
my_forward(m['BF'])
t.left(90)
my_forward(m['BE'])
t.right(180)
my_forward(m['AA']+m['BA'])
t.color('purple', 'purple')
t.pendown()
draw_letter(l['N'])
t.penup()
t.right(180)
my_forward(2*m['AA']+2*m['BA']+m['BB']+m['BC'])
t.right(180)
t.pendown()
for i in range(2):
    draw_letter(l['chevy'])
    t.penup()
    t.right(90)
    my_forward(m['AJ']+m['BB'])
    t.left(90)
    t.pendown()
draw_letter(l['chevy'])
t.penup()
t.left(90)
my_forward(2*m['AJ']+2*m['BB'])
t.right(180)
t.left(a['aa'])
my_forward(m['AK'])
t.right(180)
t.right(a['aa'])
my_forward(m['BB'])
t.left(90)
t.pendown()
draw_letter(l['grootshoofd_outer'])
t.penup()
t.right(90)
my_forward(m['BH'])
t.left(90)
t.pendown()
t.color('purple', 'purple')
draw_letter(l['grootshoofd_inner'])
t.penup()
t.forward(m['BC']*0.5+m['BB']+2*m['BA']+3*m['AA'])
t.left(90)
t.forward(m['BH']+3*m['BB']+3*m['AJ'])
t.pendown()
t.left(180)
t.color('black','black')
t.forward(m['th']/2)
t.penup()
angle = t.heading()
t.setheading(180)
t.heading()
t.forward(12)
t.right(90)
t.write(str(round(m['th']/0.6*scaling_factor*1000)/1000) + "m", font=("Arial", 2, "normal"))
t.right(90)
t.forward(12)
t.setheading(angle)
t.heading()
t.pendown()
t.forward(m['th']/2)
t.left(180)
t.penup()
t.forward(m['th'])
t.left(90)
t.pendown()


t.forward(m['tw']/2)
t.penup()
angle = t.heading()
t.setheading(270)
t.heading()
t.forward(5)
t.right(90)
t.write(str(round(m['tw']/0.6*scaling_factor*1000)/1000) + "m", font=("Arial", 2, "normal"))
t.right(90)
t.forward(5)
t.setheading(angle)
t.heading()
t.pendown()
t.forward(m['tw']/2)
t.penup()

#my_forward(1000) #my people need me!

# done()
#exitonclick()

# t.update()

t.save_as('logo_in_meters.svg')

In [79]:
omtreks = [i * scaling_factor * 1/0.6 for i in omtreks]
print(omtreks)

[5.6460566777739, 8.967266488229136, 8.967266488229136, 5.6460566777739, 7.034322378544188, 1.8399502349922003, 8.90084229202003, 4.2511485573827015, 4.2511485573827015, 4.2511485573827015, 12.61889682030694, 7.625896269982475, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
